## 0) Setup

Define paths to relevant folders.

In [1]:
MagritteFolder = '/home/frederik/Dropbox/Astro/Magritte/modules/Magritte/'
ProjectFolder  = '/home/frederik/MagritteProjects/Lines_1D_LTE/'
DataFolder     = '/home/frederik/Dropbox/Astro/Data/'

Add Magritte's `/setup/` and `/bin/` directories to the Python path.

In [2]:
from sys import path
path.insert (0, f'{MagritteFolder}setup/')
path.insert (0, f'{MagritteFolder}bin/')

Import Magritte's Python modules and setup.

In [3]:
from pyMagritte import Model, Long1, Long2, Double1, Double2, String1

from setup import Setup, linedata_from_LAMDA_file

## 1) Define model

Define helper quantities for model.

In [4]:
dimension = 1
ncells    = 20
nrays     = 2
nspecs    = 5
nlspecs   = 1
nquads    = 39

dens = 1.0E+12   # [m^-3]
abun = 1.0E+4    # [m^-3]
temp = 1.0E+02   # [K]
turb = 0.0E+00   # [m/s]
dx   = 1.0E4     # [m]
bx   = 1.3E0
dT   = 0.0E0     # [K]
dv   = 0.0E0     # [m/s]

In [5]:
setup = Setup (dimension = dimension)

Create a Magritte model object.

In [6]:
model = Model ()

Define model parameters.

In [7]:
model.parameters.set_ncells  (ncells)
model.parameters.set_nrays   (nrays)
model.parameters.set_nspecs  (nspecs)
model.parameters.set_nlspecs (nlspecs)
model.parameters.set_nquads  (nquads)

Define geometry. First define cells.

In [8]:
model.geometry.cells.x  = Double1 ([i*dx for i in range(ncells)])
model.geometry.cells.y  = Double1 ([0.0  for i in range(ncells)])
model.geometry.cells.z  = Double1 ([0.0  for i in range(ncells)])

model.geometry.cells.vx = Double1 ([0.0  for i in range(ncells)])
model.geometry.cells.vy = Double1 ([0.0  for i in range(ncells)])
model.geometry.cells.vz = Double1 ([0.0  for i in range(ncells)])

# Note that the points need to be specified before neighbors can be found
model.geometry.cells = setup.neighborLists (model.geometry.cells)

Then define the boundary of the geometry.

In [9]:
model.geometry.boundary.boundary2cell_nr = Long1 ([0, ncells-1])

Finally, define the rays for the geometry.

In [10]:
model.geometry.rays = setup.rays (nrays=nrays)

Define thermodynamics.

In [11]:
model.thermodynamics.temperature.gas   = Double1 ([temp for _ in range(ncells)])
model.thermodynamics.turbulence.vturb2 = Double1 ([turb for _ in range(ncells)])

Define the chemical species involved.

In [12]:
model.chemistry.species.abundance = Double2 ([ Double1 ([0.0, abun, dens, 0.0, 1.0]) for _ in range(ncells)])
model.chemistry.species.sym       = String1 (['dummy0', 'HCO+', 'H2', 'e-', 'dummy1'])

Define the folder containing the linedata.

In [13]:
linedataFolder = f'{DataFolder}Linedata/hco+.dat'

Define the linedata.

In [14]:
model.lines.linedata.append (linedata_from_LAMDA_file (linedataFolder, model.chemistry.species))

Define the quadrature roots and weights.

In [15]:
import quadrature

model.lines.quadrature_roots   = Double1 (quadrature.H_roots   (nquads))
model.lines.quadrature_weights = Double1 (quadrature.H_weights (nquads))

## 2) Write input file

In [16]:
from ioMagritte import IoPython
from setup      import model_name

In [17]:
modelName = f'{ProjectFolder}{model_name()}.hdf5'

Define an io object to handle input and output. (In this case via Python using HDF5.)

In [18]:
io = IoPython ("io_hdf5", modelName)

model.write (io)

0

## 3) Run the model
### Option 1: in the notebook

In [19]:
from pyMagritte import Simulation, Parameters

In [20]:
simulation = Simulation ()
parameters = Parameters ()

In [21]:
simulation.read (io)

0

In [22]:
simulation.compute_spectral_discretisation ()

0

In [23]:
simulation.compute_boundary_intensities ()

0

In [24]:
simulation.compute_LTE_level_populations ()

0

In [25]:
simulation.compute_radiation_field ()

0

In [26]:
simulation.radiation.write (io)

0

### Option 2: in the shell

In [27]:
from subprocess import Popen, PIPE

def run(command):
    # Run command pipe output
    process = Popen(command, stdout=PIPE, shell=True)
    # Continuously read output and print
    while True:
        # Extract output
        line = process.stdout.readline().rstrip()
        # Break if there is no more line
        if not line:
            break
        # Convert bytelike object to string
        line = line.decode("utf-8")
        # Output line
        print(line)

In [28]:
#run(f'{MagritteFolder}bin/example_1.exe {modelName}')

## 4) Plot the output

In [27]:
def u (r, p, f):
    index = simulation.radiation.index (p, f)
    return simulation.radiation.u[r][index]

In [28]:
from bokeh.plotting import figure, show, gridplot
from bokeh.palettes import cividis
from bokeh.io       import output_notebook
output_notebook()

Loading BokehJS ...

In [29]:
def color(s):
    ns = int((s_max-s_min) / s_step + 1)
    es = int((s    -s_min) / s_step)
    return cividis(ns)[es]

def legend(s):
    return f'{s}'

In [30]:
s_min  = 0
s_max  = ncells
s_step = 2

#### Plot of the spectrum

In [31]:
nfreqs = simulation.parameters.nfreqs()

plot = figure (width=700, height=500, y_axis_type='log')

for s in range(s_min, s_max, s_step):
    x = [simulation.radiation.frequencies.nu[s][f] for f in range(nfreqs)]
    y = [u(0, s, f)                                for f in range(nfreqs)]
    plot.line(x, y, legend=f'{s}')

plot.xaxis.axis_label = "frequencies [Hz]"
plot.yaxis.axis_label = "Mean intensity [W/m^2]"
show(plot)

#### **Analytical model**

Assuming a constant source function $S_{\nu}(x)=S_{\nu}$ along the ray and boundary condition $B_{\nu}$ on both sides of the ray, the mean intensity J and 1D flux G are given by

\begin{align}
    J_{\nu}(\tau(x)) \ &= \ S_{\nu} \ + \ \frac{1}{2} \ \left(B_{\nu}-S_{\nu}\right) \ \left[e^{-\tau_{\nu}(x)} + e^{-\tau_{\nu}(L-x)}\right], \\
    G_{\nu}(\tau(x)) \ &= \ \color{white}S_{\color{white}\nu} \ - \ \frac{1}{2} \ \left(B_{\nu}-S_{\nu}\right) \ \left[e^{-\tau_{\nu}(x)} - e^{-\tau_{\nu}(L-x)}\right],
\end{align}

where the optical depth $\tau_{\nu}$ is given by

\begin{equation}
    \tau_{\nu}(\ell) \ = \ \int_{0}^{\ell} \text{d} l \ \chi_{\nu}(l) .
\end{equation}

The frequency dependence of the opacity only comes from the line profile

\begin{equation}
    \chi_{\nu}(x) \ = \ \chi_{ij} \phi_{\nu},
\end{equation}

where we assume a Gaussian profile

\begin{equation}
	\phi_{\nu}^{ij}(x) \ = \ \frac{1}{\sqrt{\pi} \ \delta\nu_{ij}} \ \exp \left[-\left(\frac{\nu-\nu_{ij}} {\delta\nu_{ij}(x)}\right)^{2}\right], \hspace{5mm} \text{where} \hspace{5mm} \delta\nu_{ij}(x) \ = \ \frac{\nu_{ij}}{c} \sqrt{ \frac{2 k_{b} T(x)}{m_{\text{spec}}} \ + \ v_{\text{turb}}^{2}(x)}.
\end{equation}

Assuming a constant opacity along the ray, the integral for the optical depth yields

\begin{equation}
  \tau_{\nu}(\ell) \ = \ \chi_{ij} \phi_{\nu} \ell.
\end{equation}

In [40]:
line = 17

In [41]:
import numpy as np
from scipy.special import erf
import tests

linedata = simulation.lines.linedata[0]

c     = 2.99792458E+8    # [m/s] speed of light
kb    = 1.38064852E-23   # [J/K] Boltzmann's constant
mp    = 1.6726219E-27    # [kg] proton mass
T_CMB = 2.7254800        # [K] CMB temperature
vturb = 0.0E0            # [m/s] turbulent speed


pops       = tests.LTEpop (linedata, temp) * abun
emissivity = tests.lineEmissivity (linedata, pops)
opacity    = tests.lineOpacity    (linedata, pops)
source     = tests.lineSource     (linedata, pops)

def bcd (nu):
    return tests.planck(T_CMB, nu)

S    =  source[line]
chi  = opacity[line]
L    = dx * (ncells-1)
nuij = linedata.frequency[line]
dnu  = nuij/c * np.sqrt(2.0*kb*temp/mp + turb**2)


def phi (nu):
    return 1 / (np.sqrt(np.pi) * dnu) * np.exp(-((nu-nuij)/dnu)**2)

def tau (nu, l):
    return chi * phi(nu) * l
    
def J (nu, x):
    tau1 = tau(nu, x)
    tau2 = tau(nu, L-x)
    B = bcd (nu)
    return S + 0.5 * (B-S) * (np.exp(-tau1) + np.exp(-tau2))

def G (nu, x):
    tau1 = tau(nu, x)
    tau2 = tau(nu, L-x)
    B = bcd (nu)
    return   - 0.5 * (B-S) * (np.exp(-tau1) - np.exp(-tau2))

def relativeError (a,b):
    return 2.0 * np.abs((a-b)/(a+b))

In [42]:
source

array([2.39200274e-16, 9.36306579e-16, 2.06122179e-15, 3.58471227e-15,
       5.47841179e-15, 7.71482428e-15, 1.02673253e-14, 1.31101624e-14,
       1.62184545e-14, 1.95681895e-14, 2.31362216e-14, 2.69002669e-14,
       3.08388983e-14, 3.49315389e-14, 3.91584551e-14, 4.35007478e-14,
       4.79403438e-14, 5.24599848e-14, 5.70432173e-14, 6.16743801e-14])

In [43]:
chi

3.040212656325657e-09

In [44]:
simulation.lines.opacity[0]*nuij*abun

9.176612461887386e-05

In [45]:
simulation.lines.emissivity[0] / simulation.lines.opacity[0]

2.3920027446705398e-16

#### Plot of a single line

In [46]:
plot_model = figure(width=400, height=400, y_axis_type="log")
for s in range(s_min, s_max, s_step):
    M = int(simulation.lines.nr_line[s][0][line][20] - 18    )
    N = int(simulation.lines.nr_line[s][0][line][20] + 18 + 1)
    # model
    x = nuij + 18 * dnu * np.linspace(-1,1,500)
    y = J(x, simulation.geometry.cells.x[s])
    plot_model.line(x, y, color=color(s))
    # data
    x = [simulation.radiation.frequencies.nu[s][f] for f in range(M,N)]
    y = [u(0, s, f)                                for f in range(M,N)]
    plot_model.circle(x, y, color=color(s), legend=f'{s}')

# plot_error = figure(title='Line error', width=400, height=400, y_axis_type="log")
# for s in range(s_min, s_max, s_step):
#     M = int(lnr_data[0][s][line] - 18    )
#     N = int(lnr_data[0][s][line] + 18 + 1)
#     # error
#     x = nu_data[0][s][M:N]
#     y = relativeError(J(x, model.x[s]), J_data[0][s][M:N])
#     plot_error.circle(x, y, color=color(s), legend=legend(s))

#plot = gridplot([[plot_model, plot_error]])

show(plot_model)

In [1]:
from sys import path as syspath
from os  import path as  ospath

# Define directory containing Magritte
MagritteFolder = ospath.dirname(ospath.realpath('__file__')) + '/../'

# Add Magritte's bin directory to the Python path
syspath.insert(0, f'{MagritteFolder}bin/')
syspath.insert(0, f'{MagritteFolder}setup/linedata')

In [9]:
import numpy as np
import h5py  as hp

folder = "/home/frederik/Desktop/Magritte/modules/Magritte/tests/testData/model.hdf5"



In [10]:
%rm /home/frederik/Desktop/Magritte/modules/Magritte/tests/testData/model.hdf5

In [11]:
model.cells.ncells = 2

In [12]:
model.cells.x = Double1([0.1, 0.2])
model.cells.y = Double1([0.0, 0.0])
model.cells.z = Double1([0.0, 0.0])

model.cells.boundary2cell_nr = Long1([2])

In [13]:
from read_LAMDA_data import read_LAMDA_file
from pyMagritte      import Linedata, CollisionPartner

file_name    = '/home/frederik/Desktop/Magritte/modules/Magritte/tests/testData/linedata/hco+.dat'
speciesNames = ['dummy0', 'test', 'H2', 'e-', 'dummy1']

model.nlspecs = 1
model.linedata.append(read_LAMDA_file (file_name, speciesNames))

2

In [7]:
model.write (io)

linedata/lspec_0/ num
linedata/lspec_0/ sym
linedata/lspec_0/colpartner/colpar_0/ num_col_partner
linedata/lspec_0/colpartner/colpar_0/ orth_or_para_H2


0

In [15]:
cells.nei

AttributeError: 'pyMagritte.Cells' object has no attribute 'neighbor'

In [14]:
model.linedata[0].A

Double1[4.251e-05, 0.0004081, 0.001476, 0.003627, 0.007244, 0.01271, 0.0204, 0.03071, 0.044, 0.06066, 0.08108, 0.1056, 0.1347, 0.1686, 0.2078, 0.2526, 0.3034, 0.3605, 0.4245, 0.4955]

In [3]:
type(linedata.frequency)

pyMagritte.Double1

In [4]:
linedata.frequency= [1,2]


TypeError: (): incompatible function arguments. The following argument types are supported:
    1. (self: pyMagritte.Linedata, arg0: pyMagritte.Double1) -> None

Invoked with: <pyMagritte.Linedata object at 0x7f330c77ad50>, [1, 2]

In [5]:
ob = linedata.frequency

a=linedata.frequency.append
print(a)
linedata.frequency
dir(linedata.frequency)

<bound method PyCapsule.append of Double1[8.91885e+10, 1.78375e+11, 2.67558e+11, 3.56734e+11, 4.45903e+11, 5.35062e+11, 6.24209e+11, 7.13342e+11, 8.02459e+11, 8.91559e+11, 9.80639e+11, 1.0697e+12, 1.15873e+12, 1.24774e+12, 1.33672e+12, 1.42567e+12, 1.51459e+12, 1.60348e+12, 1.69233e+12, 1.78114e+12]>


['__bool__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pybind11_module_local_v2__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'count',
 'extend',
 'insert',
 'pop',
 'remove']

In [6]:
a[0] = 5
print(a[0])

TypeError: 'method' object does not support item assignment

In [7]:
linedata

In [8]:
ob

Double1[8.91885e+10, 1.78375e+11, 2.67558e+11, 3.56734e+11, 4.45903e+11, 5.35062e+11, 6.24209e+11, 7.13342e+11, 8.02459e+11, 8.91559e+11, 9.80639e+11, 1.0697e+12, 1.15873e+12, 1.24774e+12, 1.33672e+12, 1.42567e+12, 1.51459e+12, 1.60348e+12, 1.69233e+12, 1.78114e+12]

In [50]:
ob.append(0.1)

In [51]:
ob

[1.0, 2.0, 0.1]

In [52]:
linedata.frequency = [1,2, 3]

In [75]:
linedata.frequency

[1.0, 2.0, 3.0]

In [60]:
linedata.frequency

[1.0, 2.0, 3.0]

In [59]:
linedata.frequency

[1.0, 2.0, 3.0]

In [63]:
class Test:
    def __init__ (self):
        self.array = []

In [64]:
test = Test ()

In [66]:
test.array.append(0.4)

In [68]:
test.array

[0.4]

In [71]:
model.cells.x.append(0.5)

In [72]:
model.cells.x

[0.1, 0.2, 0.3]

In [74]:
model.cells.ncells = []

TypeError: (): incompatible function arguments. The following argument types are supported:
    1. (self: pyMagritte.Cells, arg0: int) -> None

Invoked with: <pyMagritte.Cells object at 0x7eff64705500>, []

In [5]:
linedata.frequency

[0.0]

In [6]:
linedata.frequency[0] = 1.0

In [11]:
linedata.frequency = [1, 2, 3]

In [12]:
linedata.frequency

[1.0, 2.0, 3.0]

In [12]:
Double1()

Double1[5]

In [8]:
def check (statement):
    assert (statement)

In [10]:
check(False)

AssertionError: 

In [5]:
import h5py  as hp
import numpy as np

In [6]:
file = hp.File ('test.hdf5')

In [10]:
syms = np.array (['Pieter', 'Jan'], dtype='S10')

In [11]:
file.create_dataset(name='sym', data=syms)

<HDF5 dataset "sym": shape (2,), type "|S10">

In [15]:
np.array(file.get('sym'))[0]

b'Pieter'